#### Web Scraping EPL's 2014/15 season statistics from <a href=https://understat.com/>understat.com</a>

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://understat.com/league'
leagues = ['EPL', 'La liga', 'Bundesliga', 'Serie A', 'Ligue 1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [3]:
url = base_url + '/' + leagues[0] + '/' + seasons[0]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

scripts = soup.find_all('script')

In [4]:
import json

string_with_json_obj = ''

for el in scripts:
    if 'teamsData' in str(el):
        string_with_json_obj = str(el).strip()

ind_star = string_with_json_obj.index("('") + 2
ind_end = string_with_json_obj.index("')")

json_data = string_with_json_obj[ind_star:ind_end].encode('utf8').decode('unicode_escape')


In [5]:
data = json.loads(json_data)

# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

print(data.keys())
print('=' * 50)
print(data['87'].keys())
print('=' * 50)

dict_keys(['71', '72', '74', '75', '76', '77', '78', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '91', '92', '202'])
dict_keys(['id', 'title', 'history'])


In [6]:
print(data['87']['id'])
print('=' * 50)
print(data['87']['title'])
print('=' * 50)
print(data['87']['history'][0])

87
Liverpool
{'h_a': 'h', 'xG': 1.33192, 'xGA': 1.55248, 'npxG': 1.33192, 'npxGA': 1.55248, 'ppda': {'att': 238, 'def': 21}, 'ppda_allowed': {'att': 401, 'def': 36}, 'deep': 6, 'deep_allowed': 9, 'scored': 2, 'missed': 1, 'xpts': 1.1776999999999997, 'result': 'w', 'date': '2014-08-17 13:30:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': -0.2205600000000001}


In [7]:
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']

print(teams)

{'71': 'Aston Villa', '72': 'Everton', '74': 'Southampton', '75': 'Leicester', '76': 'West Bromwich Albion', '77': 'Sunderland', '78': 'Crystal Palace', '80': 'Chelsea', '81': 'West Ham', '82': 'Tottenham', '83': 'Arsenal', '84': 'Swansea', '85': 'Stoke', '86': 'Newcastle United', '87': 'Liverpool', '88': 'Manchester City', '89': 'Manchester United', '91': 'Hull', '92': 'Burnley', '202': 'Queens Park Rangers'}


In [8]:
columns = []
values = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 0.909774, 0.423368, 0.909774, 0.423368, {'att': 323, 'def': 23}, {'att': 132, 'def': 32}, 4, 3, 1, 0, 1.8321999999999998, 'w', '2014-08-16 15:00:00', 1, 0, 0, 3, 0.48640599999999995]


In [9]:
mancity_data = []
for row in data['89']['history']:
    mancity_data.append(list(row.values()))

df = pd.DataFrame(mancity_data, columns=columns)
df.head()

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,1.166350,0.278076,1.166350,0.278076,"{'att': 253, 'def': 25}","{'att': 341, 'def': 19}",8,2,1,2,2.2359,l,2014-08-16 12:45:00,0,0,1,0,0.888274
1,a,0.941557,0.666489,0.941557,0.666489,"{'att': 297, 'def': 27}","{'att': 267, 'def': 21}",2,4,1,1,1.6740,d,2014-08-24 16:00:00,0,1,0,1,0.275068
2,a,0.467281,0.513746,0.467281,0.513746,"{'att': 150, 'def': 21}","{'att': 305, 'def': 27}",11,2,0,0,1.2096,d,2014-08-30 12:45:00,0,1,0,1,-0.046465
3,h,2.209110,0.563524,2.209110,0.563524,"{'att': 228, 'def': 26}","{'att': 546, 'def': 12}",9,4,4,0,2.5363,w,2014-09-14 16:00:00,1,0,0,3,1.645586
4,a,1.711910,2.542200,1.711910,1.019860,"{'att': 174, 'def': 26}","{'att': 285, 'def': 23}",7,6,3,5,0.8067,l,2014-09-21 13:30:00,0,0,1,0,0.692050


In [10]:
dataframes = {}
for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
        teams_data.append(list(row.values()))

    df = pd.DataFrame(teams_data, columns=columns)
    dataframes[team] = df
    print('Added data for {}.'.format(team))

Added data for Aston Villa.
Added data for Everton.
Added data for Southampton.
Added data for Leicester.
Added data for West Bromwich Albion.
Added data for Sunderland.
Added data for Crystal Palace.
Added data for Chelsea.
Added data for West Ham.
Added data for Tottenham.
Added data for Arsenal.
Added data for Swansea.
Added data for Stoke.
Added data for Newcastle United.
Added data for Liverpool.
Added data for Manchester City.
Added data for Manchester United.
Added data for Hull.
Added data for Burnley.
Added data for Queens Park Rangers.


In [11]:
print(dataframes['Chelsea'].head())

  h_a       xG       xGA     npxG     npxGA                     ppda  \
0   a  2.12996  0.371141  2.12996  0.371141  {'att': 206, 'def': 15}   
1   h  2.02939  0.550894  2.02939  0.550894  {'att': 134, 'def': 19}   
2   a  2.02665  1.449100  2.02665  1.449100  {'att': 307, 'def': 29}   
3   h  3.56581  1.021530  3.56581  1.021530  {'att': 344, 'def': 28}   
4   a  1.02093  1.055920  1.02093  1.055920  {'att': 313, 'def': 21}   

              ppda_allowed  deep  deep_allowed  scored  missed    xpts result  \
0  {'att': 377, 'def': 21}    10             2       3       1  2.6745      w   
1  {'att': 293, 'def': 32}    12             1       2       0  2.4703      w   
2  {'att': 167, 'def': 27}     7            11       6       3  1.7879      w   
3  {'att': 252, 'def': 17}    24             2       4       2  2.7208      w   
4  {'att': 223, 'def': 39}     2            17       1       1  1.3518      d   

                  date  wins  draws  loses  pts     npxGD  
0  2014-08-18 20:00:

In [12]:
for team, df in dataframes.items():
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

print(dataframes['Chelsea'].head(2))

  h_a       xG       xGA     npxG     npxGA                     ppda  \
0   a  2.12996  0.371141  2.12996  0.371141  {'att': 206, 'def': 15}   
1   h  2.02939  0.550894  2.02939  0.550894  {'att': 134, 'def': 19}   

              ppda_allowed  deep  deep_allowed  scored  ...    xpts  result  \
0  {'att': 377, 'def': 21}    10             2       3  ...  2.6745       w   
1  {'att': 293, 'def': 32}    12             1       2  ...  2.4703       w   

                  date wins  draws  loses  pts     npxGD  ppda_coef  \
0  2014-08-18 20:00:00    1      0      0    3  1.758819  13.733333   
1  2014-08-23 15:00:00    1      0      0    3  1.478496   7.052632   

   oppda_coef  
0   17.952381  
1    9.156250  

[2 rows x 21 columns]


In [13]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [14]:
frames = []
for team, df in dataframes.items():
    sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
    mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
    final_df = sum_data.join(mean_data)
    final_df['team'] = team
    final_df['matches'] = len(df)
    frames.append(final_df)

full_stat = pd.concat(frames)

In [15]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1, len(full_stat)+1)

In [16]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [17]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [18]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Chelsea,38,26,9,3,73,32,87,68.64,...,31.52,-0.48,29.24,35.50,10.94,13.42,407,171,75.32,-11.68
1,2,Manchester City,38,24,7,7,83,38,79,75.82,...,40.50,2.50,37.45,32.15,7.98,15.08,575,144,73.10,-5.90
2,3,Arsenal,38,22,9,7,71,36,75,69.80,...,35.72,-0.28,33.44,31.04,8.66,13.25,398,171,75.17,0.17
3,4,Manchester United,38,20,10,8,62,37,70,54.21,...,39.84,2.84,36.80,13.60,7.65,15.52,267,194,63.03,-6.97
4,5,Tottenham,38,19,7,12,58,53,64,52.39,...,57.04,4.04,51.60,-3.17,8.00,11.30,210,232,48.94,-15.06
5,6,Liverpool,38,18,8,12,52,48,62,51.70,...,38.25,-9.75,35.83,11.30,9.91,12.12,306,201,61.17,-0.83
6,7,Southampton,38,18,6,14,54,33,60,54.97,...,39.11,6.11,37.59,15.09,9.26,14.88,270,183,62.82,2.82
7,8,Swansea,38,16,8,14,46,49,56,40.90,...,55.94,6.94,54.42,-15.04,12.83,11.46,156,310,43.32,-12.68
8,9,Stoke,38,15,9,14,48,45,54,46.26,...,47.03,2.03,43.22,0.75,10.41,10.60,195,239,51.97,-2.03
9,10,Crystal Palace,38,13,9,16,47,51,48,44.76,...,45.48,-5.52,40.82,-0.77,11.58,6.58,180,258,50.56,2.56


In [19]:
full_stat.to_csv('EPL_2014-15.csv')